In [ ]:
import os
import pandas as pd

data_folder = 'data/偏心データまとめ_20241120'
reference_folder = 'data/reference_data'

if not os.path.exists(reference_folder):
    os.makedirs(reference_folder)


for root, dirs, files in os.walk(data_folder):

    relative_path = os.path.relpath(root, data_folder)
    new_folder = os.path.join(reference_folder, relative_path)

    if not os.path.exists(new_folder):
        os.makedirs(new_folder)
    
    if not any(file in ['Cut.da', 'Mz.da'] for file in files): 
        continue

    data_list = [] 
    line_counter = 0

    for file in ('Cut.da', 'Mz.da'):
        with open(os.path.join(root, file), 'r') as f:
            for line in f:
                if line.startswith('#'):
                    continue
                line = line.strip().split()      
                if len(line) >= 2:  
                    if line_counter < 480:
                        a = f"A1_{line_counter:04}"
                    elif line_counter < 960:
                        a = f"E1_{line_counter-480:04}"
                    else:
                        a = f"C1_{line_counter-960:04}"
                    data_list.append([root, a, line[1]])
                    line_counter += 1 
      
    if data_list: 
        df = pd.DataFrame(data_list, columns=['File','Area','Data'])
        combined_csv_path = os.path.join(new_folder, 'data.csv')
        df.to_csv(combined_csv_path, index=False)  

In [ ]:
reference_data = "data/reference_data"
output_csv = "data/merged_reference_data.csv"
summary_file = "画像一覧まとめ_20241004.xlsx"
summary_df = pd.read_excel(summary_file,sheet_name="ウエハ一覧")
folder_column = "ウエハフォルダ名"
number_column = "ウエハ番号"

data_frames = []

for root, dirs, files in os.walk(reference_data):
    for file in files:
        if file == "data.csv":
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            data_frames.append(df)

merged_dfs = pd.concat(data_frames, ignore_index=True)

for index, row in merged_dfs.iterrows():

    original_path = row['File']
    last_part = os.path.basename(original_path.replace("\\", "/"))

    match_rows = summary_df[summary_df[folder_column] == last_part]

    if not match_rows.empty:
        new_value = f"{match_rows[number_column].values[0].replace('-', '_')}"
        merged_dfs.at[index, 'File'] = new_value

merged_dfs.to_csv(output_csv, index=False)